In [1]:
from HestonFFT import Call_Heston
import numpy as np 
import scipy 
import math
import scipy.integrate
import pandas as pd
import torch
import torch.nn as nn
from ModelLarge import ModelLarge
from TestPerformance import test_performanceNN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#loading the training set
df = pd.read_csv("HestonTrainXXL.csv").to_numpy()[:,1:]
y = df[:,0].reshape(-1,1)
x = df[:,1:]
train_x = torch.from_numpy(x).double()
train_y = torch.from_numpy(y).reshape(-1,1).double() 

In [3]:
#setting the hyperparameters of the model 
input_dim = 8
hidden_dim = 128

model = ModelLarge(input_dim, hidden_dim).to(device)
model = model.double()

#loss and learning rate 
criterion = nn.L1Loss()
learning_rate = 0.001

#optimizer
optimizer = torch.optim.Adamax(model.parameters(), lr = learning_rate) 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                            optimizer, factor=0.5, 
                            patience=3, 
                            verbose=True ) 


#valutation set 
df = pd.read_csv("HestonTestNN.csv").to_numpy()[:,1:]
test_x = df[:,1:]
test_y = df[:,0]
test_y_Tensor = torch.from_numpy(test_y).to(device)
test_x_Tensor = torch.from_numpy(test_x).to(device)

In [6]:
train = False
if train: 
    num_epochs = 300
    iter = 0
    N = train_x.shape[0]
    batch = 100

    train_x = train_x.to(device)
    train_y = train_y.to(device)

    for epoch in range(num_epochs):
        index = torch.randperm(N)
        train_input_shuffled = train_x[index]
        train_target_shuffled = train_y[index]
        s = 0

        for b in range(0, N, batch):
            x = train_x.narrow(0, b, batch)
            y = train_y.narrow(0, b, batch)

            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()

            # Forward pass to get output/logits
            outputs = model(x)
            # print(y, outputs)
            # Calculate Loss:
            loss = criterion(outputs, y)
            # print(loss)
            s += loss

            # Getting gradients w.r.t. parameters
            loss.backward()

            # Updating parameters
            optimizer.step()

            iter += 1
        if epoch % 2 == 0:
            test = f"Test Loss:  {test_performanceNN(test_x_Tensor , test_y_Tensor , model):.2e}"
            print(f"Epoch: {epoch}    Loss: {s.mean()}    {test}")
        # scheduler wll reduce the learning rate if gor 5 epoch there is no gain in term of the loss
        scheduler.step(s.mean())


In [5]:
#saving the model
#torch.save(model, "modelNNLarge.pt")

In [4]:
#loading the model trained on 1milion observations  and look at the performance +
mod = torch.load("modelNNLarge.pt")

datasets = ["HestonTrainXXL.csv", "HestonTestS.csv", "HestonTestM.csv", "HestonTestL.csv"]
for dataset in datasets:
    print(dataset)
    df = pd.read_csv(dataset).to_numpy()[:,1:]
    X = df[:,1:]
    y = df[:,0]
    test_y_Tensor = torch.from_numpy(y).to(device)
    test_x_Tensor = torch.from_numpy(X).to(device)
    test_performanceNN(test_x_Tensor, test_y_Tensor, mod, type_ = "both", to_return = False, time_ = True)
    print("\n")


HestonTrainXXL.csv
Time:  2.792698621749878
AAE:  5.03e-05
MAE:  2.21e-03


HestonTestS.csv
Time:  0.014949798583984375
AAE:  6.09e-05
MAE:  1.46e-03


HestonTestM.csv
Time:  0.03089594841003418
AAE:  5.69e-05
MAE:  8.60e-04


HestonTestL.csv
Time:  0.08970069885253906
AAE:  5.28e-05
MAE:  9.55e-04




In [5]:
#loading the model trained on 100k observations  and look at the performance +
mod = torch.load("modelNNLarge_SmallTrain.pt")

datasets = ["HestonTrainXL.csv", "HestonTestS.csv", "HestonTestM.csv", "HestonTestL.csv"]
for dataset in datasets:
    print(dataset)
    df = pd.read_csv(dataset).to_numpy()[:,1:]
    X = df[:,1:]
    y = df[:,0]
    test_y_Tensor = torch.from_numpy(y).to(device)
    test_x_Tensor = torch.from_numpy(X).to(device)
    test_performanceNN(test_x_Tensor, test_y_Tensor, mod, type_ = "both", to_return = False, time_ = True)
    print("\n")


HestonTrainXL.csv
Time:  0.24553990364074707
AAE:  1.56e-04
MAE:  1.94e-03


HestonTestS.csv
Time:  0.009966611862182617
AAE:  1.78e-04
MAE:  2.34e-03


HestonTestM.csv
Time:  0.032892465591430664
AAE:  1.76e-04
MAE:  1.34e-03


HestonTestL.csv
Time:  0.052826642990112305
AAE:  1.71e-04
MAE:  2.67e-03


